In [9]:
import tensorflow as tf
import numpy as np
import config as cfg
import csv
import pandas as pd
from PIL import Image
import cv2

In [41]:
class image:
    filename: str
    width_height: int
    TL: list
    BR: list
    points: list

    def __init__(self, fn, tlx, tly, brx, bry, points):
        # print(f'fucking {fn}')
        self.filename = fn
        self.TL = [tlx, tly]
        self.BR = [brx, bry]
        self.points = points
        self.width_height = self.BR[0] - self.TL[0]

    def to_list(self):
        return [self.filename, self.TL, self.width_height, self.points]

    def read_all(path):
        file = open(path)
        csvreader = csv.reader(file)
        all = []
        i = 0
        for row in csvreader:
            if i == 0:
                i += 1
                continue
            ye = True
            for value in row:
                if value.startswith('-'):
                    ye = False
                    break
            if ye:
                all.append(image(row[0], int(row[1]), int(
                    row[2]), int(row[3]), int(row[5]), row[5:]).to_list())
        file.close()
        return all


In [3]:
def split_dataset(dataset, test_ratio=0.20):
    """Splits a panda dataframe in two."""
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]


In [34]:
# def crop_face(image, image_path, x, y, w, h):
#     roi_color = image[y:y + h, x:x + w]
#     if image_path.endswith('png'):
#         cv2.imwrite('data\data\croped\\' + str(w) +
#                 str(h) + '_faces.png', roi_color)
#         return roi_color, 'data\data\croped\\' + str(w) + str(h) + '_faces.png'
        
#     if image_path.endswith('jpg') or image_path.endswith('jpeg'):
#         cv2.imwrite('data\data\croped\\' + str(w) +
#                 str(h) + '_faces.jpg', roi_color)
#         return roi_color, 'data\data\croped\\' + str(w) + str(h) + '_faces.jpg'
def crop_face(image, image_path, x, y, w, h):
    roi_color = image[y:y + h, x:x + w]
    return roi_color


In [50]:
def prep_image(dp):
    image_path = dp[0]
    bb = dp[1]
    wh = dp[2]
    points = dp[3]
    # print(image_path)
    # crop out face from image
    image = cv2.imread(cfg.IMAGES_PATH + image_path)
    # image, path = crop_face(image, image_path,
    #                     int(bb[0]), int(bb[1]), int(wh), int(wh))
    image = crop_face(image, image_path, int(bb[0]), int(bb[1]), int(wh), int(wh))
    path = 'data\data\\tmp\\' + image_path
    cv2.imwrite(path, image)
    
    # resize image to xxx * xxx
    image = Image.open(path)
    image = image.resize((194, 194))

    path = 'data\data\\croped\\' + image_path
    image.save(path)

    # image = cv2.resize(image, (wh, wh), interpolation=cv2.INTER_AREA)
    # path = 'data\data\croped\\' + image_path
    # cv2.imwrite(path, image)


    # save resize ratio
    crop_r = cfg.CROP_SIZE / wh

    # resize points using resize ratio
    for point in points:
        point = float(point)
        point = point * crop_r

    # convert points to relative [0.0 -> 1]
    npo = []
    for point in points:
        point_ = float(point) / wh
        npo.append(point_)
    # for i in range(0, 135, 2):
    #     point1 = float(points[i]) / wh
    #     point2 = float(points[i+1]) / wh
    #     npo.append([point1, point2])
    

    # (should I really convert it to tensor?)
    npo = tf.convert_to_tensor(npo, dtype=tf.float32)

    # PROFIT?
    return path, npo


In [42]:
input_shape = (194, 194, 3)
output_shape = (136, )
ds = image.read_all(cfg.LABELS_PATH)
ds = pd.DataFrame(ds)
train_ds, test_ds = split_dataset(ds)


In [51]:
trainX = []
trainY = []
testX = []
testY = []

for i in range(train_ds.shape[0]):
    path, points = prep_image(train_ds.iloc[i])
    input_image = tf.io.read_file(path)
    input_image = tf.image.decode_image(input_image)
    input_image = tf.cast(input_image, dtype=tf.int32)
    trainX.append(input_image)
    trainY.append(points)

    # VERY TEMPOPARY BREAK
    # break

for i in range(test_ds.shape[0]):
    path, points = prep_image(train_ds.iloc[i])
    input_image = tf.io.read_file(path)
    input_image = tf.image.decode_image(input_image)
    input_image = tf.cast(input_image, dtype=tf.int32)
    testX.append(input_image)
    testY.append(points)

    # VERY TEMPOPARY BREAK
    # break

In [63]:
trainX_ = tf.convert_to_tensor(trainX, dtype = tf.int32)
trainY_ = tf.convert_to_tensor(trainY, dtype = tf.float32)

testX_ = tf.convert_to_tensor(testX, dtype=tf.int32)
testY_ = tf.convert_to_tensor(testY, dtype=tf.float32)


In [53]:
import tensorflow as tf
from keras_vggface.vggface import VGGFace


In [66]:
vgg_model = VGGFace(include_top=False, model= 'vgg16',input_shape=(
    cfg.CROP_SIZE, cfg.CROP_SIZE, 3))
last_layer = vgg_model.get_layer('pool5').output
x = tf.keras.layers.Flatten(name='flatten')(last_layer)
x = tf.keras.layers.Dense(2048, activation='relu', name='fc6')(x)
x = tf.keras.layers.Dense(512, activation='relu', name='fc7')(x)
out = tf.keras.layers.Dense(136, activation='relu', name='fc8')(x)
model = tf.keras.Model(vgg_model.input, out)
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 194, 194, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 194, 194, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 194, 194, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 97, 97, 64)        0         
                                                                 
 conv2_1 (Conv2D)            (None, 97, 97, 128)       73856     
                                                                 
 conv2_2 (Conv2D)            (None, 97, 97, 128)       147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 48, 48, 128)       0   

In [58]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsolutePercentageError(
                  name='mean_absolute_percentage_error'
              ),
              metrics=['accuracy'])


In [ ]:
model.fit(trainX_, trainY_, epochs=10,
          validation_data=(testX_, testY_))


In [73]:
from smallervggnet import SmallerVGGNet
import os

In [69]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
p_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)
model = SmallerVGGNet.build()
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsolutePercentageError(
                  name='mean_absolute_percentage_error'
              ),
              metrics=['accuracy'])


In [ ]:
model.fit(trainX_, 
          trainY_,
          epochs=100,
          validation_data=(testX_, testY_),
          callbacks=[cp_callback])
model.save('saved_model/my_model')


In [ ]:
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(testX_, testY_, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))
